<a href="https://colab.research.google.com/github/kplr-training/Web-Scraping/blob/main/Solution/3_Web_Scraping_avec_Scrapy.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Web Scraping à l'aide de  Scrapy**

Nous allons scrapper **quotes.toscrape.com** , un site qui répertorie des citations d'auteurs celebres.

![image](https://user-images.githubusercontent.com/123748165/224308580-0d85356c-f4d4-4aa4-a7df-c2a04c12e155.png)

**Ce Workshop vous guidera à travers ces taches :**

1.Creer un nouveau projet Scrapy

2.Ecrire une araignee pour explorer un site et extraire des donnees

3.Exporter les donnees recuperees à l'aide de la ligne de commande

4.Changer l'araignee pour suivre recursivement les liens

5.Utiliser des arguments d'araignee

**Installer Scrapy :**

In [ ]:
# install scrapy
!pip install Scrapy

**Creer un projet**

- Avant de commencer à scraper, vous devrez configurer un nouveau projet Scrapy.

- Créer un repertoire dans lequel vous souhaitez stocker et executez votre code  :

In [ ]:
# create files for learning
!scrapy startproject firstproject

New Scrapy project 'firstproject', using template directory '/usr/local/lib/python3.8/dist-packages/scrapy/templates/project', created in:
    /content/firstproject

You can start your first spider with:
    cd firstproject
    scrapy genspider example example.com


- Cela créera un repertoire 'firstproject' avec le contenu suivant :

![image](https://user-images.githubusercontent.com/123748165/224308606-e2a63a79-3b68-4e94-b044-3bf24718c866.png)

In [ ]:
# get the current working directory
import os
os.getcwd()

'/content'

In [ ]:
# change working directories
os.chdir('/content/firstproject/firstproject/spiders')
os.getcwd()

'/content/firstproject/firstproject/spiders'

# **1 - Extraire les quotes :**

**Notre premier Spider :**

- Les spiders sont des classes que vous definissez et que Scrapy utilise pour extraire des informations d'un site Web (ou d'un groupe de sites Web).


In [ ]:
# create quotes_spider.py and save it under the firstproject/firstproject/spiders directory
%%writefile -a quotes_spider.py
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com/page/1/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
        
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

Writing quotes_spider.py



- **name**: identifie le spider. Il doit etre unique au sein d'un projet, c'est-à -dire que vous ne pouvez pas definir le meme nom pour differentes spiders.


- **response.css()** : Le resultat de l'execution response.css('div.quote')est un objet de type liste appelé SelectorList, qui represente une liste d' Selector objets qui entourent les elements XML/HTML et vous permettent d'executer d'autres requetes pour affiner la selection ou extraire les donnees

- **yield** : Un Spider Scrapy genere generalement de nombreux dictionnaires contenant les donnees extraites de la page. Pour ce faire, nous utilisons le yield pour specifier ce qu'on veut extraire'



![image](https://user-images.githubusercontent.com/123748165/224308623-ad6a2c7a-105b-4739-8cf4-96a681ad4015.png)

-  la methode **parse()**  recherche le lien vers la page suivante, construit un URL absolue complet à  l'aide de la méthode urljoin()  (puisque les liens peuvent etre relatifs) et genere une nouvelle requete vers la page suivante, s'enregistrant comme rappel à  gerer l'extraction des donnees pour la page suivante et de continuer à  parcourir toutes les pages.

![image](https://user-images.githubusercontent.com/123748165/224308645-1c24ccf1-cf48-48be-abee-ac5f18a0c745.png)

Comment faire fonctionner notre Spider :

- Pour faire fonctionner notre araignee, accedez au repertoire de niveau superieur du projet et executez :
- Cette commande execute l'araignee avec le nom quotes que nous venons d'ajouter, qui enverra des requetes pour le quotes.toscrape.com domaine.

In [ ]:
!scrapy crawl quotes

**Stockage des donnees scrapees**

- Le moyen le plus simple de stocker les donnees extraites consiste à  utiliser Feed exports , avec la commande suivante : !scrapy crawl quotes -O quotes.json

- Cela generera un quotes.json fichier contenant tous les elements recuperes, serialises en JSON .

In [ ]:
!scrapy crawl quotes -O quotes.json

**Convertir nos données au format CSV :**

In [ ]:
import pandas as pd
df = pd.read_json('/content/firstproject/firstproject/spiders/quotes.json')
df.to_csv('data.csv')
df.head(10)

,text,author,tags
0,â€œThe world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"â€œIt is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,â€œThere are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"â€œThe person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"â€œImperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
5,â€œTry not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
6,â€œIt is better to be hated for what you are tha...,AndrÃ© Gide,"[life, love]"
7,"â€œI have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
8,â€œA woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]
9,"â€œA day without sunshine is like, you know, nig...",Steve Martin,"[humor, obvious, simile]"


# **2 - Extraire des donnees sur les Auteurs :**

**On va suivre faire la meme procedure pour extraire les donnees des auteurs :**

In [ ]:
%%writefile -a author_spider.py
import scrapy


class AuthorSpider(scrapy.Spider):
    name = 'author'

    start_urls = ['https://quotes.toscrape.com/']

    def parse(self, response):
        author_page_links = response.css('.author + a')
        yield from response.follow_all(author_page_links, self.parse_author)

        pagination_links = response.css('li.next a')
        yield from response.follow_all(pagination_links, self.parse)

    def parse_author(self, response):
        def extract_with_css(query):
            return response.css(query).get(default='').strip()

        yield {
            'name': extract_with_css('h3.author-title::text'),
            'birthdate': extract_with_css('.author-born-date::text'),
            'bio': extract_with_css('.author-description::text'),
        }

Writing author_spider.py


In [ ]:
!scrapy crawl author

In [ ]:
!scrapy crawl author -O authors.json

In [ ]:
df = pd.read_json('/content/firstproject/firstproject/spiders/authors.json')
df.to_csv('data.csv')
df.head(10)

,name,birthdate,bio
0,Albert Einstein,"March 14, 1879","In 1879, Albert Einstein was born in Ulm, Germ..."
1,AndrÃ© Gide,"November 22, 1869",AndrÃ© Paul Guillaume Gide was a French author ...
2,Marilyn Monroe,"June 01, 1926",Marilyn Monroe (born Norma Jeane Mortenson; Ju...
3,Eleanor Roosevelt,"October 11, 1884",Anna Eleanor Roosevelt was an American politic...
4,J.K. Rowling,"July 31, 1965",See also: Robert GalbraithAlthough she writes ...
5,Steve Martin,"August 14, 1945","Stephen Glenn ""Steve"" Martin is an American ac..."
6,Jane Austen,"December 16, 1775",Jane Austen was an English novelist whose work...
7,Thomas A. Edison,"February 11, 1847","Thomas Alva Edison was an American inventor, s..."
8,Allen Saunders,"April 24, 1899","Allen Saunders was an American writer, journal..."
9,Mark Twain,"November 30, 1835","Samuel Langhorne Clemens, better known by his ..."
